# Fine Tuning RoBERTa
In this notebook, we fine-tuned RoBERTa, using the base cased version.

### Import Libraries
We used scikit-learn for dataset splitting, and the Hugging Face `transformers` library to download the model and perform training.

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
import transformers
import torch
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [3]:
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)

2024-06-17 18:22:33.668960: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Read the Dataset
In this case we use the original version of the dataset

In [4]:
dataRew=pd.read_json("../Dataset/IMDB_reviews.json",lines=True)

In [5]:
dataRew.drop(columns=["movie_id","rating","review_date","user_id","review_summary"],inplace=True)

Change the  Dataset in a suitable  form

In [6]:
dataRew['is_spoiler'] = dataRew['is_spoiler'].map({True: 1, False: 0})
dataRew = dataRew.rename(columns={'is_spoiler': 'label'})

## Split the Dataset

In [7]:
train, test,= train_test_split(dataRew, test_size=0.2, stratify=dataRew['label'],random_state=42)

#### Divide the Dataset in Valuation and Training

In [8]:
train, val,= train_test_split(train, test_size=0.2, stratify=train['label'],random_state=42)

In [9]:
train['label'].value_counts()

label
0    270713
1     96591
Name: count, dtype: int64

In [10]:
val['label'].value_counts()

label
0    67678
1    24148
Name: count, dtype: int64

In [11]:
test['label'].value_counts()

label
0    84598
1    30185
Name: count, dtype: int64

### Change the Dataset

In [12]:
Train = Dataset.from_pandas(train)
Eval = Dataset.from_pandas(val)
Test = Dataset.from_pandas(test)

In [13]:
Train=Train.remove_columns("__index_level_0__")
Eval=Eval.remove_columns("__index_level_0__")
Test=Test.remove_columns("__index_level_0__")

### Tokenization

In [14]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

/home/f.caprari/prova/Group10venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
def encodeBig(text):
    return tokenizer(text['review_text'], padding="max_length", truncation=True, max_length=512)

In [16]:
Train=Train.map(encodeBig,batched=True)

Map:   0%|          | 0/367304 [00:00<?, ? examples/s]

In [17]:
Eval=Eval.map(encodeBig,batched=True)

Map:   0%|          | 0/91826 [00:00<?, ? examples/s]

In [18]:
Test=Test.map(encodeBig,batched=True)

Map:   0%|          | 0/114783 [00:00<?, ? examples/s]

### Apply the Model

https://huggingface.co/bhavyagiri/roberta-base-finetuned-imdb-spoilers?text=Jack+Ryan+is+so+amazing

In [19]:
BATCH_SIZE = 16
WEIGHT_DECAY=0.01
LR = 2e-5
EPOCHS = 5


In [20]:

model = RobertaForSequenceClassification.from_pretrained("roberta-base")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:


training_args = TrainingArguments(
output_dir="test_dirRob",
learning_rate=LR,
weight_decay=WEIGHT_DECAY,
num_train_epochs=EPOCHS,
evaluation_strategy="epoch",
per_device_train_batch_size=BATCH_SIZE,
per_device_eval_batch_size=BATCH_SIZE,
report_to="tensorboard",
save_strategy='no',
fp16=True

)

In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metricsweighted(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
    'accuracy': acc,
    'f1': f1,
    'precision': precision,
    'recall': recall
    }

In [23]:
def compute_metricsbinary(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
    'accuracy': acc,
    'f1': f1,
    'precision': precision,
    'recall': recall
    }

In [24]:
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Train,
    eval_dataset=Eval,
    compute_metrics=compute_metricsbinary,
)

In [ ]:
history=trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
log_history=trainer.state.log_history

In [ ]:
log_history

In [ ]:
 for log in log_history:
     if 'eval_loss' in log:
        print(list(log.keys()))
        

In [ ]:
with open("../Output/outputRoB.txt", "a") as f:
    for log in log_history:
        if 'eval_loss' in log:
            print(f"Epoch: {log.get('epoch')}, Eval Loss: {log['eval_loss']}, Accuracy: {log['eval_accuracy']}, F1: {log['eval_f1']}, Precision: {log['eval_precision']}, Recall: {log['eval_recall']}",file=f)

In [ ]:
log_history=trainer.evaluate(Test)

In [ ]:
log_history

In [ ]:
with open("../Output/outputRoB.txt", "a") as f:
    print("Result on Test",file=f)
    print(f"Eval Loss: {log_history['eval_loss']}, Accuracy: {log_history['eval_accuracy']}, F1: {log_history['eval_f1']}, Precision: {log_history['eval_precision']}, Recall: {log_history['eval_recall']}",file=f)

# ROBERTA, apply to the clean Dataset

## Read the clean Dataset

In [ ]:
CleanData=pd.read_csv("../Dataset/datiClean.csv")
CleanData=CleanData[["clean_review","is_spoiler"]]

In [ ]:
CleanData["clean_review"] = CleanData["clean_review"].apply(safe_literal_eval)
CleanData["whole__text"] = CleanData["clean_review"].apply(join_tokens)

In [ ]:
CleanData['is_spoiler_numeric'] = np.where(CleanData['is_spoiler'] == True, 1, 0)

In [ ]:
CleanData = CleanData.rename(columns={'is_spoiler_numeric': 'label','whole__text':'text'})
CleanData = CleanData[['text','label']]

In [ ]:
train, test,= train_test_split(CleanData, test_size=0.2, stratify=CleanData['label'],random_state=42)

train, val,= train_test_split(train, test_size=0.2, stratify=train['label'],random_state=42)

### Tokenize the Dataset

In [ ]:
Train = Dataset.from_pandas(train)
Eval = Dataset.from_pandas(val)
Test = Dataset.from_pandas(test)

Train=Train.remove_columns("__index_level_0__")
Eval=Eval.remove_columns("__index_level_0__")
Test=Test.remove_columns("__index_level_0__")

In [ ]:
def encodeBig(text):
    return tokenizer(text['text'], padding="max_length", truncation=True, max_length=512)

In [ ]:
Train=Train.map(encodeBig,batched=True)

In [ ]:
Eval=Eval.map(encodeBig,batched=True)

In [ ]:
Test=Test.map(encodeBig,batched=True)

### Train the model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Train,
    eval_dataset=Eval,
    compute_metrics=compute_metricsbinary,
)

In [ ]:
history=trainer.train()

In [ ]:
ROBERTAChistory=trainer.state.log_history

In [ ]:
for log in log_history:
         if 'eval_loss' in log:
            print(f"Epoch: {log.get('epoch')}, Eval Loss: {log['eval_loss']}, Accuracy: {log['eval_accuracy']}, F1: {log['eval_f1']}, Precision: {log['eval_precision']}, Recall: {log['eval_recall']}")

In [ ]:
with open("../Output/outputROBERTaClean.txt", "a") as f:
    for log in log_history:
         if 'eval_loss' in log:
            print(f"Epoch: {log.get('epoch')}, Eval Loss: {log['eval_loss']}, Accuracy: {log['eval_accuracy']}, F1: {log['eval_f1']}, Precision: {log['eval_precision']}, Recall: {log['eval_recall']}",file=f)

In [ ]:
log_history=trainer.evaluate(Test)

In [ ]:
with open("../Output/outputROBERTaClean.txt", "a") as f:
    print("Result on Test",file=f)
    print(f"Eval Loss: {log_history['eval_loss']}, Accuracy: {log_history['eval_accuracy']}, F1: {log_history['eval_f1']}, Precision: {log_history['eval_precision']}, Recall: {log_history['eval_recall']}",file=f)